<a href="https://colab.research.google.com/github/batbakamaxno/testmaxno/blob/master/%D0%B3%D0%B8%D0%B3%D0%B0%D1%87%D0%B0%D1%82_%D1%81%D0%BA%D1%80%D0%B8%D0%BD%D0%BA%D0%B0%D1%81%D1%82_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Работа с API через библиотеку requests
Для начала работы нужно [получить доступ](https://developers.sber.ru/docs/ru/gigachat/api/integration).
## Авторизация
Объявим ключи для авторизации

In [8]:
# Используем вкладку "Секреты" левой панели Colab для хранения ключей
from google.colab import userdata
client_id = userdata.get('SBER_ID')
secret = userdata.get('SBER_SECRET')
auth = userdata.get('SBER_AUTH')

Для авторизации можно использовать либо пару client_id и secret, но это требует некоторых действий

In [ ]:
import base64
credentials = f"{client_id}:{secret}"
encoded_credentials = base64.b64encode(credentials.encode('utf-8')).decode('utf-8')

И получаемая в итоге строка идентична нашему ключу auth

In [ ]:
encoded_credentials == auth

True

Поэтому используем для авторизации auth.

[Документация по авторизации](https://developers.sber.ru/docs/ru/gigachat/api/reference/rest/post-token)

In [9]:
import requests
import uuid

def get_token(auth_token, scope='GIGACHAT_API_PERS'):
    """
      Выполняет POST-запрос к эндпоинту, который выдает токен.

      Параметры:
      - auth_token (str): токен авторизации, необходимый для запроса.
      - область (str): область действия запроса API. По умолчанию — «GIGACHAT_API_PERS».

      Возвращает:
      - ответ API, где токен и срок его "годности".
      """
    # Создадим идентификатор UUID (36 знаков)
    rq_uid = str(uuid.uuid4())

    # API URL
    url = "https://ngw.devices.sberbank.ru:9443/api/v2/oauth"

    # Заголовки
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded',
        'Accept': 'application/json',
        'RqUID': rq_uid,
        'Authorization': f'Basic {auth_token}'
    }

    # Тело запроса
    payload = {
        'scope': scope
    }

    try:
        # Делаем POST запрос с отключенной SSL верификацией
        # (можно скачать сертификаты Минцифры, тогда отключать проверку не надо)
        response = requests.post(url, headers=headers, data=payload, verify=False)
        return response
    except requests.RequestException as e:
        print(f"Ошибка: {str(e)}")
        return -1


In [10]:
response = get_token(auth)
if response != 1:
  print(response.text)
  giga_token = response.json()['access_token']

/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ngw.devices.sberbank.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


{"access_token":"eyJjdHkiOiJqd3QiLCJlbmMiOiJBMjU2Q0JDLUhTNTEyIiwiYWxnIjoiUlNBLU9BRVAtMjU2In0.Bl6i_uvlQq9hxChfHKnHCDpoMWNLXcRnES84zB7hRzRnAL6HFkZPnH5HPRY-iHieNuodzZVJ8rqy01QJadZnriqpFf5YIBJ4p7Ww7Efq3w8OGTn6M9LXYODZWZb3DOOw1CE_2Llugf7YZWIJddNw8g4M_M5OZ6OMN2ZJplYaeC_-5d2SUTJlORRGOolUGc5WY7jqiceGwxaBMbG9rd65uNX-UdLc1qdtfnNLt1PrOUIRg50DRMWIxDhW4IiwKS1NqMyipDc8xtCiKgrA6Rntgf0-52pcC1MhlZHaPHkS9Cn9FxQDX29LD47GziD6KXQ7niG-yGqDRNqi8qz6INA1aQ.az_zql8Wt9YJxohHfYkoFw.kOYXppsiBMRxJfYPsSO_NptCou8gvwc_d-ahXdKYgDE6xhJ-qNaK-oh0I_tb0oBk_JlN__9vtn0Q4VTrJ-bNS5PnBKBwx3wANMqfML5Uru-akw-o-CPFFkbJg0M09cd39A0vWHzC6Qdg74W4h4Dli6Ngl1ltpssdSS51Osqdu1keW5yD-00vTeii6SpT5LR2_QO-uZ7Ro9WHAFyFYDQ6iMGO6XYztvI3rkJWuMFTspSVpbvjmT9CbReGFvWJCHONTMWzndYDa2vrsyG8rc8qtfgmEj25aSXsHZvmCCkenFtbBluPEB1ndeePFm3mK5CHdYueAykDaDQCAgQYljCUrHe1xZ05n6BREvCzoxp2BkjE4bWJC4LP_FspPZWvWftRb3j5w6FodJ4I5BhgbuLYLJK_GQ7OmPKRrwjQABiF0HsIKgLO_d-bksgU4MKK8Iu-XuJuwSI4vaZSRN95gtYFLNzSoy47mpMgeTKV8KZKyowuJbcqqxxkfR1zHZR3DdDnEOnnO0poA2tUf792beoO_clrytpl4

## Получение перечня моделей
[Документация](https://developers.sber.ru/docs/ru/gigachat/api/reference/rest/get-models)

In [11]:
import requests

url = "https://gigachat.devices.sberbank.ru/api/v1/models"

payload={}
headers = {
  'Accept': 'application/json',
  'Authorization': f'Bearer {giga_token}'
}

response = requests.request("GET", url, headers=headers, data=payload, verify=False)

print(response.text)

/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gigachat.devices.sberbank.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


{"object":"list","data":[{"id":"GigaChat","object":"model","owned_by":"salutedevices","type":"chat"},{"id":"GigaChat-Max","object":"model","owned_by":"salutedevices","type":"chat"},{"id":"GigaChat-Plus","object":"model","owned_by":"salutedevices","type":"chat"},{"id":"GigaChat-Pro","object":"model","owned_by":"salutedevices","type":"chat"}]}


## Сделаем запросы на генерацию текста
Начнем с отправки отдельных сообщений

In [14]:
import requests
import json

def get_chat_completion(auth_token, user_message):
    """
    Отправляет POST-запрос к API чата для получения ответа от модели GigaChat.

    Параметры:
    - auth_token (str): Токен для авторизации в API.
    - user_message (str): Сообщение от пользователя, для которого нужно получить ответ.

    Возвращает:
    - str: Ответ от API в виде текстовой строки.
    """
    # URL API, к которому мы обращаемся
    url = "https://gigachat.devices.sberbank.ru/api/v1/chat/completions"

    # Подготовка данных запроса в формате JSON
    payload = json.dumps({
        "model": "GigaChat",  # Используемая модель
        "messages": [
            {
                "role": "user",  # Роль отправителя (пользователь)
                "content": user_message  # Содержание сообщения
            }
        ],
        "temperature": 1,  # Температура генерации
        "top_p": 0.1,  # Параметр top_p для контроля разнообразия ответов
        "n": 1,  # Количество возвращаемых ответов
        "stream": False,  # Потоковая ли передача ответов
        "max_tokens": 512,  # Максимальное количество токенов в ответе
        "repetition_penalty": 1,  # Штраф за повторения
        "update_interval": 0  # Интервал обновления (для потоковой передачи)
    })

    # Заголовки запроса
    headers = {
        'Content-Type': 'application/json',  # Тип содержимого - JSON
        'Accept': 'application/json',  # Принимаем ответ в формате JSON
        'Authorization': f'Bearer {auth_token}'  # Токен авторизации
    }

    # Выполнение POST-запроса и возвращение ответа
    try:
        response = requests.request("POST", url, headers=headers, data=payload, verify=False)
        return response
    except requests.RequestException as e:
        # Обработка исключения в случае ошибки запроса
        print(f"Произошла ошибка: {str(e)}")
        return -1



In [22]:
answer = get_chat_completion(giga_token, 'Как на Java сделать GET запрос?')

/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gigachat.devices.sberbank.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [21]:
answer.json()

{'choices': [{'message': {'content': 'Чтобы сделать GET запрос на Java, можно использовать библиотеку `java.net.HttpURLConnection` или библиотеку `org.apache.http`. Я покажу пример с использованием `HttpURLConnection`, так как это стандартный способ для Java.\n\n### Пример с использованием `HttpURLConnection`:\n\n```java\nimport java.io.BufferedReader;\nimport java.io.InputStreamReader;\nimport java.net.HttpURLConnection;\nimport java.net.URL;\n\npublic class HttpGetExample {\n    public static void main(String[] args) throws Exception {\n        // Создаем URL для запроса\n        URL url = new URL("https://example.com/api/endpoint");\n\n        // Открываем соединение\n        HttpURLConnection connection = (HttpURLConnection) url.openConnection();\n        connection.setRequestMethod("GET");\n\n        // Получаем ответ\n        BufferedReader reader = new BufferedReader(new InputStreamReader(connection.getInputStream()));\n        String response;\n        while ((response = reader

In [ ]:
print(answer.json()['choices'][0]['message']['content'])

In [23]:
from IPython.display import display, Markdown
display(Markdown(answer.json()['choices'][0]['message']['content']))

На Java для выполнения HTTP GET запросов можно использовать различные библиотеки, такие как `HttpClient`, `JDK` или сторонние библиотеки, например, `OkHttp`. Я покажу пример на основе `HttpClient`, который входит в стандартную библиотеку Java.

### Пример кода:

```java
import java.net.URI;
import java.net.http.HttpClient;
import java.net.http.HttpRequest;
import java.net.http.HttpResponse;
import java.util.concurrent.CompletionStage;

public class HttpGetExample {
    public static void main(String[] args) {
        HttpClient client = HttpClient.newHttpClient();
        HttpRequest request = HttpRequest.newBuilder()
                .uri(URI.create("https://example.com")) // Замените на нужный URL
                .build();

        CompletionStage<HttpResponse<String>> response = client.send(request, HttpResponse.BodyHandlers.ofString());

        response.thenAccept(response -> {
            System.out.println(response.body()); // Вывод ответа на консоль
        });
    }
}
```

### Пояснение:

1. **HttpClient**: Это класс из стандартной библиотеки Java, который позволяет отправлять HTTP-запросы и получать ответы.

2. **HttpRequest.newBuilder()**: Создает объект запроса, который содержит URL и другие параметры запроса.

3. **.uri(URI.create("https://example.com"))**: Указывает URL, к которому будет отправлен запрос.

4. **.build()**: Превращает объект запроса в полноценный запрос.

5. **client.send(request, HttpResponse.BodyHandlers.ofString())**: Отправляет запрос и получает ответ в виде строки.

6. **response.thenAccept(response -> {...})**: Обрабатывает ответ, выводя его на консоль.

### Альтернативные библиотеки:

Если вам нужно что-то более мощное или удобное, можно использовать сторонние библиотеки, такие как:

- **OkHttp**: Очень популярная библиотека для HTTP-запро

## Создадим диалог

In [25]:
import requests
import json

def get_chat_completion(auth_token, user_message, conversation_history=None):
    """
    Отправляет POST-запрос к API чата для получения ответа от модели GigaChat в рамках диалога.

    Параметры:
    - auth_token (str): Токен для авторизации в API.
    - user_message (str): Сообщение от пользователя, для которого нужно получить ответ.
    - conversation_history (list): История диалога в виде списка сообщений (опционально).

    Возвращает:
    - response (requests.Response): Ответ от API.
    - conversation_history (list): Обновленная история диалога.
    """
    # URL API, к которому мы обращаемся
    url = "https://gigachat.devices.sberbank.ru/api/v1/chat/completions"

    # Если история диалога не предоставлена, инициализируем пустым списком
    if conversation_history is None:
        conversation_history = []

    # Добавляем сообщение пользователя в историю диалога
    conversation_history.append({
        "role": "user",
        "content": user_message
    })

    # Подготовка данных запроса в формате JSON
    payload = json.dumps({
        "model": "GigaChat:latest",
        "messages": conversation_history,
        "temperature": 1,
        "top_p": 0.1,
        "n": 1,
        "stream": False,
        "max_tokens": 512,
        "repetition_penalty": 1,
        "update_interval": 0
    })

    # Заголовки запроса
    headers = {
        'Content-Type': 'application/json',
        'Accept': 'application/json',
        'Authorization': f'Bearer {auth_token}'
    }

    # Выполнение POST-запроса и возвращение ответа
    try:
        response = requests.post(url, headers=headers, data=payload, verify=False)
        response_data = response.json()
        print(response_data)

        # Добавляем ответ модели в историю диалога
        conversation_history.append({
            "role": "assistant",
            "content": response_data['choices'][0]['message']['content']
        })

        return response, conversation_history
    except requests.RequestException as e:
        # Обработка исключения в случае ошибки запроса
        print(f"Произошла ошибка: {str(e)}")
        return None, conversation_history



In [26]:
# Пример использования функции для диалога

conversation_history = []

# Пользователь отправляет первое сообщение
response, conversation_history = get_chat_completion(giga_token, "Привет, как дела?", conversation_history)

# Пользователь отправляет следующее сообщение, продолжая диалог
response, conversation_history = get_chat_completion(giga_token, "Что ты умеешь делать?", conversation_history)


/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gigachat.devices.sberbank.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


{'choices': [{'message': {'content': 'Привет! У меня всё отлично, готов помочь тебе с любыми вопросами. Как у тебя дела?', 'role': 'assistant'}, 'index': 0, 'finish_reason': 'stop'}], 'created': 1739720597, 'model': 'GigaChat:1.0.26.20', 'object': 'chat.completion', 'usage': {'prompt_tokens': 17, 'completion_tokens': 23, 'total_tokens': 40, 'precached_prompt_tokens': 0}}


/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gigachat.devices.sberbank.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


{'choices': [{'message': {'content': 'Я могу помочь с множеством задач! Вот несколько примеров того, чем я могу быть полезен:\n\n1. **Ответы на вопросы**: я могу предоставить информацию по различным темам, будь то наука, история, культура или что-то другое.\n\n2. **Анализ текста**: могу помочь с анализом текста, его редактированием, написанием эссе или статей.\n\n3. **Математика и логика**: могу решать математические задачи, объяснять сложные концепции и помогать с логическими задачами.\n\n4. **Программирование**: могу помочь с написанием и отладкой кода, объяснением алгоритмов и технологий.\n\n5. **Генерация контента**: могу придумывать идеи, помогать с написанием текстов, стихов, рассказов и даже сценариев.\n\n6. **Обучение**: могу объяснять сложные темы простым языком, помогать с изучением языков, математики, программирования и других предметов.\n\n7. **Разговоры**: могу просто поболтать на любые темы, будь то философия, наука, искусство или что-то другое.\n\nЕсли у тебя есть конкре

In [27]:
conversation_history

[{'role': 'user', 'content': 'Привет, как дела?'},
 {'role': 'assistant',
  'content': 'Привет! У меня всё отлично, готов помочь тебе с любыми вопросами. Как у тебя дела?'},
 {'role': 'user', 'content': 'Что ты умеешь делать?'},
 {'role': 'assistant',
  'content': 'Я могу помочь с множеством задач! Вот несколько примеров того, чем я могу быть полезен:\n\n1. **Ответы на вопросы**: я могу предоставить информацию по различным темам, будь то наука, история, культура или что-то другое.\n\n2. **Анализ текста**: могу помочь с анализом текста, его редактированием, написанием эссе или статей.\n\n3. **Математика и логика**: могу решать математические задачи, объяснять сложные концепции и помогать с логическими задачами.\n\n4. **Программирование**: могу помочь с написанием и отладкой кода, объяснением алгоритмов и технологий.\n\n5. **Генерация контента**: могу придумывать идеи, помогать с написанием текстов, стихов, рассказов и даже сценариев.\n\n6. **Обучение**: могу объяснять сложные темы прост

## Добавим системный промпт
Системный промпт определяет роль, задавать контект и особенности общения бота с пользователем.

In [28]:
conversation_history = [{
    'role': 'system',
    'content': 'Отвечай как мастер джедай. Пусть тебя зовут Йода.'

}

]

In [29]:
response, conversation_history = get_chat_completion(giga_token, 'Привет, друг!', conversation_history)

/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gigachat.devices.sberbank.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


{'choices': [{'message': {'content': 'Приветствую тебя, мой друг. Как могу я помочь тебе сегодня?', 'role': 'assistant'}, 'index': 0, 'finish_reason': 'stop'}], 'created': 1739720624, 'model': 'GigaChat:1.0.26.20', 'object': 'chat.completion', 'usage': {'prompt_tokens': 36, 'completion_tokens': 17, 'total_tokens': 53, 'precached_prompt_tokens': 0}}


In [30]:
response.json()['choices'][0]['message']['content']

'Приветствую тебя, мой друг. Как могу я помочь тебе сегодня?'

## Генерация картинок Кандинским
[Документация](https://developers.sber.ru/docs/ru/gigachat/api/reference/rest/get-file-id)

Делаем запрос на генерацию прямо в GigaChat

In [ ]:
import requests
import json

def send_chat_request(giga_token, user_message):
    """
    Отправляет POST-запрос к API GigaChat для получения ответа от модели чата.

    Параметры:
    - giga_token (str): Токен авторизации для доступа к API GigaChat.
    - user_message (str): Сообщение пользователя, которое будет обработано моделью GigaChat.

    Возвращает:
    - str: Строка сгенерированного ответа GigaChat с тэгом img
    """
    # URL API для отправки запросов к GigaChat
    url = "https://gigachat.devices.sberbank.ru/api/v1/chat/completions"

    # Заголовки для HTTP-запроса
    headers = {
        'Content-Type': 'application/json',  # Указываем, что отправляемые данные в формате JSON
        'Authorization': f'Bearer {giga_token}',  # Используем токен авторизации для доступа к API
    }

    # Данные для отправки в теле запроса
    payload = {
        "model": "GigaChat:latest",  # Указываем, что хотим использовать последнюю версию модели GigaChat
        "messages": [
            {
                "role": "user",  # Роль отправителя - пользователь
                "content": user_message  # Сообщение от пользователя
            },
        ],
        "temperature": 0.7  # Устанавливаем температуру, чтобы управлять случайностью ответов
    }

    try:
        # Отправляем POST-запрос к API и получаем ответ
        response = requests.post(url, headers=headers, data=json.dumps(payload), verify=False)
        # Выводим текст ответа. В реальных условиях следует обрабатывать ответ и проверять статус коды.
        print(response.json())
        return response.json()["choices"][0]["message"]["content"]
    except requests.RequestException as e:
        # В случае возникновения исключения в процессе выполнения запроса, выводим ошибку
        print(f"Произошла ошибка: {str(e)}")
        return None





In [ ]:
user_message = "Нарисуй фотореалистичное изображение антропоморфного робота с \
ноутбуком в руках"
response_img_tag = send_chat_request(giga_token, user_message)
print(response_img_tag)

Получаем само изображение и сохраняем его в файл

In [ ]:
from bs4 import BeautifulSoup



# Парсим HTML
soup = BeautifulSoup(response_img_tag, 'html.parser')

# Извлекаем значение атрибута `src`
img_src = soup.img['src']

print(img_src)

In [ ]:
import requests

headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {giga_token}',
}

response = requests.get(f'https://gigachat.devices.sberbank.ru/api/v1/files/{img_src}/content', headers=headers, verify=False)

In [ ]:
with open('image.jpg', 'wb') as f:
    f.write(response.content)